## Кейс региональной аптечной сети

### Контекст

В связи с переходом с УСН на ОСНО у региональной аптечной сети возникла необходимость однократной загрузки данных в систему 1С. Для успешного осуществления этой операции требуется предварительное сопоставление информации о продажах из аптек с данными, полученными из системы СБИС. Все выявленные расхождения будут устранены в ручном режиме, после чего подготовленный файл будет импортирован в 1С.

### Постановка задачи

**Работа с выгрузкой из СБИС:**  
 - Загрузить все csv файлы в один датафрейм. Если встретился файл с другим расширением, пропустить.  
 - Изменить и преобразовать наименования колонок в соответствии с заданным условием.  
 

**Работа с выгрузкой из аптек:**  
 - Загружать на обработку каждый файл отдельно, содержимое каждого файла загрузить в датафрейм. Если встретился файл с другим расширением, пропустить.  
 - в каждом файле изменить номера накладных для "Поставщика" - ЕАПТЕКА.  
 - в каждом файле создать столбцы: "Номер счет-фактуры", "Сумма счет-фактуры", "Дата счет-фактуры", "Сравнение дат".  
 
**Каждый файл с данными о продажах аптек необходимо сопоставить с выгрузкой из СБИС:**  
 - Из выгрузки СБИС необходимо отобрать все записи, в которых номера накладных совпадают с номерами накладных в данных о продажах аптек.   
 - Из найденных строк необходимо оставить только те, которые имеют один из типов документа: ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]
 - В файл с данными о продажах аптек записать данные в столбцы "Номер счет-фактуры", "Сумма счет-фактуры", "Дата счет-фактуры" из отфильтрованной на предыдущем шаге выгрузки из СБИС. Если по заданным условиям находится несколько подходящих документов - выбирать значения из первого найденного. Это означает, что если у документов разные суммы, берется та, что стоит первой в выгрузке из СБИС
     - Дату представить в формате 25.05.2021. 
     - В столбеце "Сравнение дат" указать "Не совпадает!", если найденная дата и дата накладной отличаются. Иначе - пустая строка. 
 - Изменить порядок следования столбцов в полученном датафрейме в соответствии с заданным условием.    
 - Сохранить датафрейм в файл Excel по пути:  "Результат/{полная сегодняшняя дата}/{имя исходного файла без расширения} - результат.xlsx". Если таких папок не существует - создать их (проверку и создание реализовать автоматически средствами Python)

Файлы:  
    - Папка СБИС: https://disk.yandex.ru/d/xKJ4KlXJIJEULw  
    - Папка Аптеки: https://disk.yandex.ru/d/m9MtFWt56fDW7g  

In [1]:
# Загружаем необходимые библиотеки
import pandas as pd
import glob
import os
from datetime import datetime
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 80)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
# Путь к папке, содержащей CSV-файлы с выгрузкой из СБИС
file_path = '/Users/79169/Входящие'

# Поиск всех файлов с расширением .csv в указанной директории
csv_entry = glob.glob(os.path.join(file_path, '*.csv'))

df = []
# Цикл для обработка каждого файла
for file in csv_entry:
    # Загрузка содержимого CSV-файла в DataFrame
    data = pd.read_csv(file, encoding="windows-1251", sep = ';')
    df.append(data)

# Объединение всех DataFrame из списка в один DataFrame
df_sbis = pd.concat(df, axis = 0)
df_sbis.head(3)

,Дата,Номер,Сумма,Статус,Примечание,Комментарий,Контрагент,ИНН/КПП,Организация,ИНН/КПП.1,Тип документа,Имя файла,Дата.1,Номер.1,Сумма.1,Сумма НДС,Ответственный,Подразделение,Код,Дата.2,Время,Тип пакета,Идентификатор пакета,Запущено в обработку,Получено контрагентом,Завершено,Увеличение суммы,НДС,Уменьшение суммы,НДС.1
0,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,ЭДОНакл,DP_TOVTORGPR_2BE911035223f7011e2addf0050569171...,30.09.21,БРН00709545,"4 056,46","368,77",NaN,Аптека №1,3,30.09.21,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,30.09.21 23:40,01.10.21 08:39,01.10.21 14:59,"0,00","0,00","0,00","0,00"
1,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,СчФктр,ON_NSCHFDOPPR_2BE911035223f7011e2addf005056917...,30.09.21,БРН00709545,"4 056,46","368,77",NaN,Аптека №1,3,30.09.21,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"
2,30.09.21,БРН00709545,"0,00",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,ЭДОРеестрСертификатов,SE_RSERT_3255510243325701001_40254198734025010...,NaN,NaN,"0,00","0,00",NaN,Аптека №1,3,30.09.21,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"


Переименовать столбцы полученного датафрейма следующим образом:

In [4]:
colnames = ["Дата","Номер","Сумма","Статус","Примечание","Комментарий","Контрагент","ИНН/КПП","Организация","ИНН/КПП 1",
            "Тип документа","Имя файла","Дата 1","Номер 1","Сумма 1","Сумма НДС","Ответственный","Подразделение",
            "Код","Дата 2","Время","Тип пакета","Идентификатор пакета","Запущено в обработку","Получено контрагентом",
            "Завершено","Увеличение суммы","НДC","Уменьшение суммы","НДС 1"]

Все названия из нескольких слов сцепить через нижнее подчеркивание в одно_длинное_слово.

In [5]:
new_colunm_names = [col.replace(' ', '_') for col in colnames]
df_sbis.columns = new_colunm_names

In [6]:
# Путь к папке, содержащей CSV-файлы с выгрузкой из аптек
file_path = '/Users/79169/Аптеки/csv/correct'

# Поиск всех файлов с расширением .csv в указанной директории
csv_apteki = glob.glob(os.path.join(file_path, '*.csv'))

# Создаем дату текущего дня
today_date = datetime.today().date()

# Определяем путь назначения, куда будут сохраняться готовые файлы
dest_folder = f'Результат/{today_date}/'

# Создаем папку, включая родительские директории, если их нет
try:
    os.makedirs(dest_folder, exist_ok=True)
    print(f"Directory '{dest_folder}' created successfully.")
except FileExistsError:
    print(f"Directory '{dest_folder}' already exists.")

for file in csv_apteki:
    df_apteka = pd.read_csv(file, encoding="windows-1251", sep = ';')
    
    # Создайте столбцы: "Номер счет-фактуры", "Сумма счет-фактуры", "Дата счет-фактуры", "Сравнение дат"
    df_apteka['Номер счет-фактуры'], df_apteka['Сумма счет-фактуры'], \
                                df_apteka['Дата счет-фактуры'], df_apteka['Сравнение дат'] = ["", "", "", ""]
    
    # Если "Поставщик" - ЕАПТЕКА, то к "Номер накладной" добавить /15.
    df_apteka.loc[df_apteka['Поставщик'].str.contains('ЕАПТЕКА'), 'Номер накладной'] += '/15'
       

    for index, row in df_apteka.iterrows():
        # Фильтруем строки в df_filtered, где 'Номер накладной' совпадает с 'Номер накладной' в df_apteka
        matched_rows = df_sbis[df_sbis['Номер'].isin([row['Номер накладной']]) &
                                   df_sbis['Тип_документа'].isin(["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"])]

        if not matched_rows.empty:
            # Обновляем данные в df_apteka
            df_apteka.loc[index, 'Номер счет-фактуры'] = matched_rows.iloc[0]['Номер']
            df_apteka.loc[index, 'Сумма счет-фактуры'] = matched_rows.iloc[0]['Сумма']
            df_apteka.loc[index, 'Дата счет-фактуры'] = pd.to_datetime(matched_rows.iloc[0]['Дата']).strftime('%d.%m.%Y')

            # Сравниваем даты
            if pd.to_datetime(row['Дата накладной']).strftime('%d.%m.%Y') != pd.to_datetime(matched_rows.iloc[0]['Дата']).strftime('%d.%m.%Y'):
                df_apteka.loc[index, 'Сравнение дат'] = "Не совпадает!"
            else:
                df_apteka.loc[index, 'Сравнение дат'] = ""
    
    # Изменяем порядок следования колонок
    new_column_order = ['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик', 'Дата приходного документа',
                        'Номер приходного документа', 'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
                        'Сумма счет-фактуры', 'Кол-во', 'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
                        'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат']

    df_apteka = df_apteka[new_column_order]
    
    # Записываем итоговый датафрейм в Excel файл по указанной директории
    out_path = f'Результат/{today_date}/{Path(file).stem} - результат.xlsx'  

    with pd.ExcelWriter(out_path, engine='xlsxwriter') as writer:
        df_apteka.to_excel(writer, sheet_name='Sheet1')



Directory 'Результат/2025-07-26/' created successfully.
